# Exploratory Data Analysis

Chapter 2

Allen Downey

[MIT License](https://en.wikipedia.org/wiki/MIT_License)

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')

import utils
from utils import decorate
from distribution import Pmf, Cdf

## Loading and validation

Reading data from the [General Social Survey](https://gssdataexplorer.norc.org/projects/29853)

TODO: Add link to project

In [5]:
tables = pd.read_html('https://www.cdc.gov/brfss/annual_data/2017/llcp_varlayout_17_onecolumn.html')

In [9]:
layout = tables[1]
layout.index = layout['Variable Name']
layout

,Starting Column,Variable Name,Field Length
Variable Name,,,
_STATE,1,_STATE,2
FMONTH,17,FMONTH,2
IDATE,19,IDATE,8
IMONTH,19,IMONTH,2
IDAY,21,IDAY,2
IYEAR,23,IYEAR,4
DISPCODE,32,DISPCODE,4
SEQNO,36,SEQNO,10
_PSU,36,_PSU,10


In [43]:
names = ['SEX', 'INCOME2', 'WTKG3', 'HTM4', '_LLCPWT', '_AGEG5YR']

In [55]:
colspecs = []
for name in names:
    start, _, length = layout.loc[name]
    colspecs.append((start-1, start+length-1))
    
colspecs

[(124, 125),
 (179, 181),
 (2039, 2044),
 (2036, 2039),
 (1797, 1807),
 (2027, 2029)]

In [81]:
filename = 'LLCP2017.ASC.gz'
brfss = pd.read_fwf(filename,
                    colspecs=colspecs, 
                    names=names,
                    compression='gzip',
                    nrows=None)

brfss.head()

,SEX,INCOME2,WTKG3,HTM4,_LLCPWT,_AGEG5YR
0,2,6.0,7348.0,165.0,79.425947,11
1,1,8.0,9571.0,180.0,89.694580,10
2,1,99.0,8845.0,188.0,440.121376,11
3,2,1.0,7711.0,170.0,194.867164,11
4,2,2.0,6350.0,165.0,169.087888,10


In [82]:
brfss.shape

(450016, 6)

In [83]:
brfss['SEX'].value_counts().sort_index()

1    198725
2    251007
9       284
Name: SEX, dtype: int64

In [84]:
brfss['SEX'].replace([9], np.nan, inplace=True)

In [85]:
brfss['INCOME2'].value_counts().sort_index()

1.0      18346
2.0      19334
3.0      27735
4.0      34222
5.0      39751
6.0      53148
7.0      59632
8.0     122763
77.0     33328
99.0     38426
Name: INCOME2, dtype: int64

In [86]:
brfss['INCOME2'].replace([77, 99], np.nan, inplace=True)

In [92]:
brfss['WTKG3'].describe()

count    420145.000000
mean       8158.671851
std        2070.102388
min        2268.000000
25%        6759.000000
50%        7938.000000
75%        9163.000000
max       27500.000000
Name: WTKG3, dtype: float64

In [93]:
brfss['HTM4'].describe()

count    433572.000000
mean        169.821863
std          10.614508
min          91.000000
25%         163.000000
50%         170.000000
75%         178.000000
max         236.000000
Name: HTM4, dtype: float64

In [94]:
brfss['_LLCPWT'].describe()

count    450016.000000
mean        568.098034
std        1119.545360
min           0.131624
25%          91.499675
50%         237.083009
75%         592.876546
max       45053.592700
Name: _LLCPWT, dtype: float64

In [95]:
brfss['_AGEG5YR'].describe()

count    450016.000000
mean          7.747962
std           3.557474
min           1.000000
25%           5.000000
50%           8.000000
75%          11.000000
max          14.000000
Name: _AGEG5YR, dtype: float64

In [96]:
brfss['_AGEG5YR'].replace([14], np.nan, inplace=True)

In [149]:
lower = np.arange(15, 85, 5)
upper = lower + 4

In [150]:
lower[1]= 18
lower = pd.Series(lower, index=range(len(lower)))
lower

0     15
1     18
2     25
3     30
4     35
5     40
6     45
7     50
8     55
9     60
10    65
11    70
12    75
13    80
dtype: int64

In [151]:
upper[-1] = 99
upper = pd.Series(upper, index=range(len(upper)))
upper

0     19
1     24
2     29
3     34
4     39
5     44
6     49
7     54
8     59
9     64
10    69
11    74
12    79
13    99
dtype: int64

In [152]:
age_code = brfss['_AGEG5YR']

In [153]:
lower = lower[age_code]
lower.describe()

count    443915.000000
mean         53.191994
std          17.754445
min          18.000000
25%          40.000000
50%          55.000000
75%          65.000000
max          80.000000
dtype: float64

In [154]:
upper = upper[age_code]
upper.describe()

count    443915.000000
mean         58.520854
std          19.708977
min          24.000000
25%          44.000000
50%          59.000000
75%          69.000000
max          99.000000
dtype: float64

In [157]:
def randint(lower, upper):
    for low, high in zip(lower, upper+1):
        try:
            yield np.random.randint(low, high)
        except ValueError:
            yield np.nan

In [158]:
brfss['AGE'] = list(randint(lower, upper))

In [159]:
brfss['AGE'].describe()

count    443915.000000
mean         55.857849
std          18.774884
min          18.000000
25%          42.000000
50%          58.000000
75%          69.000000
max          99.000000
Name: age, dtype: float64

In [160]:
sample = utils.resample_rows_weighted(brfss, '_LLCPWT')[:100000]

In [163]:
sample.to_hdf('brfss.hdf5', 'brfss')

In [164]:
%time brfss = pd.read_hdf('brfss.hdf5', 'brfss')

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.92 ms


In [165]:
brfss.shape

(100000, 7)

In [166]:
brfss.describe()

,SEX,INCOME2,WTKG3,HTM4,_LLCPWT,_AGEG5YR,age
count,99938.000000,83356.000000,93438.000000,95195.000000,100000.000000,98816.000000,98816.000000
mean,1.511617,5.792840,8119.824119,170.217501,2779.550579,6.099893,47.707254
std,0.499868,2.228355,2087.694402,10.834020,3983.603004,3.588736,18.928562
min,1.000000,1.000000,2268.000000,91.000000,4.218451,1.000000,18.000000
25%,1.000000,4.000000,6577.000000,163.000000,576.673886,3.000000,32.000000
50%,2.000000,6.000000,7893.000000,170.000000,1383.001815,6.000000,47.000000
75%,2.000000,8.000000,9072.000000,178.000000,3195.533502,9.000000,62.000000
max,2.000000,8.000000,27216.000000,226.000000,45053.592700,13.000000,99.000000
